In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error
import numpy as np
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import xgboost as xgb

X_path= os.path.join("..", "data","enriched_input", "X_train.csv")
X = pd.read_csv(X_path, delimiter=',')

y_path= os.path.join("..", "data","enriched_input", "y_train.csv")
y = pd.read_csv(y_path, delimiter=',')

X.set_index("DELIVERY_START", inplace=True)
y.set_index("DELIVERY_START", inplace=True)
X.index = pd.to_datetime(X.index, utc=True)
y.index = pd.to_datetime(y.index, utc=True)

In [49]:
def decompose_y(y):
    """
    Décompose les écarts en direction et magnitude.
    
    :param y: Numpy array ou une liste des écarts réels.
    :return: Tuple de deux arrays - direction (1 pour positif, 0 pour négatif), magnitude (valeur absolue de l'écart).
    """
    y_direction = (y > 0).astype(int)
    y_magnitude = np.abs(y)
    return y_direction, y_magnitude


In [50]:
y_direction, y_magnitude = decompose_y(y['spot_id_delta'])

# Division en jeux d'entraînement et de test
X_train, X_test, y_dir_train, y_dir_test, y_mag_train, y_mag_test = train_test_split(X, y_direction, y_magnitude, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)  # Augmentation de max_iter pour assurer la convergence
clf.fit(X_train, y_dir_train)
y_dir_pred = clf.predict(X_test)

# Évaluation de la performance de la classification
accuracy = accuracy_score(y_dir_test, y_dir_pred)
print(f'Accuracy de la classification: {accuracy}')

# Étape 2: Régression pour prédire l'ampleur sur les données correctement prédites
# Filtrage de X_test et y_mag_test en fonction des prédictions correctes de direction
correct_preds_index = y_dir_test == y_dir_pred  # Index des prédictions correctes
#X_test_filtered = X_test[correct_preds_index]
#y_mag_test_filtered = y_mag_test[correct_preds_index]

# Entraînement du modèle de régression sur l'ensemble d'entraînement complet (pour cet exemple)
reg = LinearRegression()
reg.fit(X_train, y_mag_train)

# Prédiction de l'ampleur sur l'ensemble filtré
y_mag_pred = reg.predict(X_test)

# Évaluation de la performance de la régression
mse = mean_squared_error(y_mag_test, y_mag_pred)
print(f'MSE de la régression: {mse}')

Accuracy de la classification: 0.5464403583215465
MSE de la régression: 563.647296943993


In [51]:
def reconstruct_y(y_direction, y_magnitude):
    """
    Reconstruit les écarts à partir de la direction et de la magnitude.
    
    :param y_direction: Numpy array ou une liste indiquant la direction des écarts.
    :param y_magnitude: Numpy array ou une liste indiquant l'ampleur des écarts.
    :return: Numpy array des écarts reconstruits.
    """
    sign_factors = y_direction * 2 - 1  # Convertit 0 en -1 et 1 reste 1
    y_reconstructed = y_magnitude * sign_factors
    return y_reconstructed

In [52]:
y_test = reconstruct_y(y_dir_test, y_mag_test)
y_pred = reconstruct_y(y_dir_pred, y_mag_pred)

In [53]:
def weighted_accuracy_score(y_true_reconstructed, y_pred_simulated):
    """
    Calcule le score de précision pondérée en utilisant les écarts réels et prédits reconstruits.
    
    :param y_true_reconstructed: Numpy array des écarts réels reconstruits.
    :param y_pred_simulated: Numpy array des écarts prédits.
    :return: Score de précision pondérée.
    """
    # Calcul de la direction correcte (1 pour correct, 0 pour incorrect)
    correct_direction = ((y_true_reconstructed * y_pred_simulated) > 0).astype(int)
    
    # Calcul des poids basés sur l'ampleur de l'écart réel
    weights = np.abs(y_true_reconstructed)
    weights /= np.sum(weights)  # Normalisation des poids
    
    # Calcul du score de précision pondérée
    weighted_accuracy = np.sum(correct_direction * weights)
    
    return weighted_accuracy

In [54]:
score = weighted_accuracy_score(y_test, y_pred)
score

0.5716691613619274

Weighted Accuracy Score: 0.5739121623696769

In [55]:
X_test_path= os.path.join("..", "data","enriched_input", "X_test.csv")
X_test = pd.read_csv(X_test_path, delimiter=',')

delivery_start = X_test["DELIVERY_START"]

In [56]:
y_direction, y_magnitude = decompose_y(y['spot_id_delta'])

In [57]:
X_test.set_index("DELIVERY_START", inplace=True)

X_test.index = pd.to_datetime(X_test.index, utc=True)

In [58]:
y_dir_pred = clf.predict(X_test)
y_mag_pred = reg.predict(X_test)

In [59]:
y_pred = reconstruct_y(y_dir_pred, y_mag_pred)

In [60]:
y_pred

array([  1.65137852,  -0.79148243,  -4.10669415, ..., -27.3306063 ,
       -24.05702052, -24.87482423])

In [61]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../data/output_y_test/y_test_result_logistic_n_regression1.csv", index = False)